##TTC Data EDA

*Purpose: To identify the TTC major delay time and location.

*Steps: refer to google docs - "https://docs.google.com/document/d/17sF-iA_EJvkyvtBuirbvTvgXYxEQt1xQMp3QbYp2m6E/edit"


# Start here

In [1]:
# For downloading files from Toronto Open Data
import requests 

# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## TTC Delay Clean

### TTC_Subway_delay_data Cleaning

In [2]:
## Create a file paths for all files
file_paths = ["ttc-subway-delay-jan-2014-april-2017.xlsx",
              "ttc-subway-delay-may-december-2017.xlsx",  
              "ttc-subway-delay-data-2018.xlsx",
              "ttc-subway-delay-data-2019.xlsx",
              "ttc-subway-delay-data-2020.xlsx",
              "ttc-subway-delay-data-2021.xlsx",
              "ttc-subway-delay-data-2022.xlsx",
              "ttc-subway-delay-data-2023.xlsx"]

# Read each file into a DataFrame and store them in a list
dataframes = [pd.read_excel(fp) for fp in file_paths]

# Concatenate all the DataFrames into one
TTC_delay_raw = pd.concat(dataframes, ignore_index=True)

TTC_delay_raw.head()

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,2014-01-01,00:21,Wednesday,VICTORIA PARK STATION,MUPR1,55,60,W,BD,5111
1,2014-01-01,02:06,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001
2,2014-01-01,02:40,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0
3,2014-01-01,03:10,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116
4,2014-01-01,03:20,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386


### TTC Delay code - Import data and some cleaning

In [3]:
## Import the TTC delay code name dataset

TTC_delay_code_name_raw = pd.read_excel("ttc-subway-delay-codes.xlsx", index_col=0)
TTC_delay_code_name_raw.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
NaN,NaN,SUB RMENU CODE,CODE DESCRIPTION,NaN,NaN,SRT RMENU CODE,CODE DESCRIPTION
NaN,1.0,EUAC,Air Conditioning,NaN,1.0,ERAC,Air Conditioning
NaN,2.0,EUAL,Alternating Current,NaN,2.0,ERBO,Body
NaN,3.0,EUATC,ATC RC&S Equipment,NaN,3.0,ERCD,Consequential Delay (2nd Delay Same Fault)
NaN,4.0,EUBK,Brakes,NaN,4.0,ERCO,Couplers


Reorignize ttc-delay-code dataset 

In [4]:
## Remove the first row since it is all NaN
TTC_delay_code_name_raw = TTC_delay_code_name_raw.iloc[1:]

## Since the code abbreviation and code describtion are divided into four different columns, seperate them into two dataframe and concat into one
TTC_delay_code_name_raw_1 = pd.DataFrame({
    "Code": TTC_delay_code_name_raw.iloc[:,1],
    "Code description": TTC_delay_code_name_raw.iloc[:,2]
})
TTC_delay_code_name_raw_2 = pd.DataFrame({
    "Code": TTC_delay_code_name_raw.iloc[:,5],
    "Code description": TTC_delay_code_name_raw.iloc[:,6]
})

TTC_delay_code_name = pd.concat([TTC_delay_code_name_raw_1, TTC_delay_code_name_raw_2], axis=0)

## Reset the index
TTC_delay_code_name = TTC_delay_code_name.reset_index(drop=True)

## Take a look
TTC_delay_code_name.head()

,Code,Code description
0,EUAC,Air Conditioning
1,EUAL,Alternating Current
2,EUATC,ATC RC&S Equipment
3,EUBK,Brakes
4,EUBO,Body


### Combine two datasets

In [5]:
## Add the column "code duscription" into TTC 2023 delay data 

TTC_delay_raw = pd.merge(TTC_delay_raw, TTC_delay_code_name, on="Code", how='left')

TTC_delay_raw.head()

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Code description
0,2014-01-01,00:21,Wednesday,VICTORIA PARK STATION,MUPR1,55,60,W,BD,5111,Priority One - Train in Contact With Person
1,2014-01-01,02:06,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001,Disorderly Patron
2,2014-01-01,02:40,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0,NaN
3,2014-01-01,03:10,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116,Disorderly Patron
4,2014-01-01,03:20,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386,Unsanitary Vehicle


## Add columns to the data frame 

### Add columns (Date/time) to be the index

In [6]:
# Convert 'HH:MM' format to timedelta and Add seconds to match 'HH:MM:SS' format
TTC_delay_raw['Time'] = TTC_delay_raw['Time'] + ":00"

# Add the 'time' timedelta to the DatetimeIndex
TTC_delay_raw.index = TTC_delay_raw["Date"] + pd.to_timedelta(TTC_delay_raw['Time'])

## Drop the "Time" and "Date" columns
TTC_delay_raw = TTC_delay_raw.drop(columns='Time')
TTC_delay_raw = TTC_delay_raw.drop(columns='Date')

## Rename the index
TTC_delay_raw.index.name = "Date/Time"

TTC_delay_raw.head()

,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Code description
Date/Time,,,,,,,,,
2014-01-01 00:21:00,Wednesday,VICTORIA PARK STATION,MUPR1,55,60,W,BD,5111,Priority One - Train in Contact With Person
2014-01-01 02:06:00,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001,Disorderly Patron
2014-01-01 02:40:00,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0,NaN
2014-01-01 03:10:00,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116,Disorderly Patron
2014-01-01 03:20:00,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386,Unsanitary Vehicle


### Add time_inetrval column 

*can drop this column if not necessary

In [7]:
## Create a function to capture the what time interval does each datetime index falls

def get_hour_interval(hour):
    start_hour = f"{hour:02d}:00"
    end_hour = f"{(hour + 1) % 24:02d}:00"  # Use modulo 24 for the hour after 23:00 to wrap around to 00:00
    return f"{start_hour} - {end_hour}"

In [8]:
## Create a new column for time interval

TTC_delay_raw['time_interval'] = TTC_delay_raw.index.hour.map(get_hour_interval)
TTC_delay_raw.head()

,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Code description,time_interval
Date/Time,,,,,,,,,,
2014-01-01 00:21:00,Wednesday,VICTORIA PARK STATION,MUPR1,55,60,W,BD,5111,Priority One - Train in Contact With Person,00:00 - 01:00
2014-01-01 02:06:00,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001,Disorderly Patron,02:00 - 03:00
2014-01-01 02:40:00,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0,NaN,02:00 - 03:00
2014-01-01 03:10:00,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116,Disorderly Patron,03:00 - 04:00
2014-01-01 03:20:00,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386,Unsanitary Vehicle,03:00 - 04:00


#### !!!The columns for Year, months, periods are added later.

### Filter the not real reasons for delay 

Replace 0 value with NaN and Remove the rows with both NaN, for columns "Min delay" and "Min Gap"
This step will filter our the delay reasons that not "real". For exampple, if we see the delay code "MUSC" which is code description "Miscellaneous Speed Control", which is the delay cause by the over-speed. This is not the delay reason that we are concern about. 


In [36]:
## Replace 0 value with NaN for columns "Min delay" and "Min Gap"
TTC_delay_raw ["Min Delay"] = TTC_delay_raw ["Min Delay"].replace(0, np.nan)
TTC_delay_raw ["Min Gap"] = TTC_delay_raw ["Min Gap"].replace(0, np.nan)

## Drop the rows where both min delay and min gap is 0 (NaN)
TTC_delay = TTC_delay_raw.dropna(subset=["Min Delay","Min Gap"], how='all')
TTC_delay.head()

,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Code description,time_interval
Date/Time,,,,,,,,,,
2014-01-01 00:21:00,Wednesday,VICTORIA PARK STATION,MUPR1,55.0,60.0,W,BD,5111,Priority One - Train in Contact With Person,00:00 - 01:00
2014-01-01 02:06:00,Wednesday,HIGH PARK STATION,SUDP,3.0,7.0,W,BD,5001,Disorderly Patron,02:00 - 03:00
2014-01-01 03:10:00,Wednesday,LANSDOWNE STATION,SUDP,3.0,8.0,W,BD,5116,Disorderly Patron,03:00 - 04:00
2014-01-01 03:20:00,Wednesday,BLOOR STATION,MUSAN,5.0,10.0,S,YU,5386,Unsanitary Vehicle,03:00 - 04:00
2014-01-01 08:48:00,Wednesday,KIPLING STATION,MUNOA,5.0,NaN,E,BD,0,No Operator Immediately Available - Not E.S.A....,08:00 - 09:00


### Total number of delay reasons 

In [23]:
##Group by the TTC delay data by the delay reason - Code, 
TTC_delay_code = TTC_delay.groupby("Code")

## Get the number of delay reasons
num_groups = TTC_delay_code.ngroups
print("There are {} reasons of delay".format(num_groups))

There are 194 reasons of delay


### Total delay by Year, Months, Days, Time_Interval 

### Total delay by Year

In [60]:
## Total delay time by year

TTC_delay.groupby(TTC_delay.index.year)["Min Delay"].sum().sort_values(ascending=False)

Date/Time
2022    73106.0
2023    53186.0
2014    41177.0
2016    38275.0
2015    35499.0
2017    14316.0
2018     5228.0
2019     4569.0
2021     4472.0
2020     3857.0
Name: Min Delay, dtype: float64

In [61]:
## Total number of delay by year

TTC_delay.groupby(TTC_delay.index.year)["Min Delay"].count().sort_values(ascending=False)

Date/Time
2022    8976
2014    6280
2023    6204
2016    5698
2015    5059
2017    2153
2018     707
2019     671
2021     561
2020     548
Name: Min Delay, dtype: int64

### Total Delay by Month

In [62]:
## Total delay time by months

TTC_delay.groupby(TTC_delay.index.month)["Min Delay"].sum().sort_values(ascending=False)

Date/Time
1     44328.0
2     26185.0
3     25843.0
4     24963.0
5     24351.0
6     21750.0
7     20329.0
9     20277.0
8     19739.0
10    15600.0
12    15342.0
11    14978.0
Name: Min Delay, dtype: float64

In [63]:
## Total number of delay by months

TTC_delay.groupby(TTC_delay.index.month)["Min Delay"].count().sort_values(ascending=False)

Date/Time
1     6128
2     3381
3     3374
4     3225
5     3224
9     2864
6     2812
7     2785
8     2631
10    2264
11    2093
12    2076
Name: Min Delay, dtype: int64

### Delay by periods

In [64]:
## Create a columns for period.
## Set five periods for a day
## Period#1 (0-6): early morning - off-rush-hour
## Period#2 (6-9): daytime - morning rush hour, Period#3 (9-16): daytime - off-rush-hour
## Period#4 (16-20): night - night rush hour, Period#5 (20 - 24): nighttime - off-rush-hour

bins = [0, 6, 9, 16, 20, 24]
labels = [1, 2, 3, 4, 5]
TTC_delay['Period'] = np.digitize(TTC_delay.index.hour, bins, right=False)
TTC_delay.loc[:, 'Period'] = TTC_delay ['Period'].map(dict(zip(range(1, len(labels)+1), labels)))

TTC_delay.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_30468\4088844673.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TTC_delay['Period'] = np.digitize(TTC_delay.index.hour, bins, right=False)
C:\Users\DELL\AppData\Local\Temp\ipykernel_30468\4088844673.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TTC_delay.loc[:, 'Period'] = TTC_delay ['Period'].map(dict(zip(range(1, len(labels)+1), labels)))


,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Code description,time_interval,Period
Date/Time,,,,,,,,,,,
2014-01-01 00:21:00,Wednesday,VICTORIA PARK STATION,MUPR1,55.0,60.0,W,BD,5111,Priority One - Train in Contact With Person,00:00 - 01:00,1
2014-01-01 02:06:00,Wednesday,HIGH PARK STATION,SUDP,3.0,7.0,W,BD,5001,Disorderly Patron,02:00 - 03:00,1
2014-01-01 03:10:00,Wednesday,LANSDOWNE STATION,SUDP,3.0,8.0,W,BD,5116,Disorderly Patron,03:00 - 04:00,1
2014-01-01 03:20:00,Wednesday,BLOOR STATION,MUSAN,5.0,10.0,S,YU,5386,Unsanitary Vehicle,03:00 - 04:00,1
2014-01-01 08:48:00,Wednesday,KIPLING STATION,MUNOA,5.0,NaN,E,BD,0,No Operator Immediately Available - Not E.S.A....,08:00 - 09:00,2


In [65]:
## Total delay time by periods

TTC_delay.groupby("Period")["Min Delay"].sum().sort_values(ascending=False)

Period
3    85863.0
4    63964.0
5    48882.0
2    43002.0
1    31974.0
Name: Min Delay, dtype: float64

In [66]:
## Total delay time by periods

TTC_delay.groupby("Period")["Min Delay"].count().sort_values(ascending=False)

Period
3    11523
4     8876
2     6611
5     6151
1     3696
Name: Min Delay, dtype: int64

### Delay by subway line

The following is the information for subway line

YU = Line 1 Yonge - University
BD = Line 2 Bloor–Danforth
SRT = Line 3 Scarborough Rapid Transit
SHP (Unknown)


In [19]:
## Sumed up delay time by subway line

TTC_delay.groupby("Line")["Min Delay"].sum().sort_values(ascending=False)

Line
YU              126386.0
BD              106736.0
SRT              28606.0
SHP              11900.0
66                  12.0
BD LINE 2            8.0
YU/BD                8.0
25 DON MILLS         3.0
29 DUFFERIN          0.0
Name: Min Delay, dtype: float64

In [56]:
# Count the total number of delay by subway line

TTC_delay.groupby("Line")["Min Delay"].count().sort_values(ascending=False)

Line
YU              18241
BD              14534
SRT              2525
SHP              1545
YU/BD               2
25 DON MILLS        1
66                  1
BD LINE 2           1
29 DUFFERIN         0
Name: Min Delay, dtype: int64

### Average delay time for each delay reasons

In [25]:
TTC_delay_code_avg_delaytime = TTC_delay.groupby("Code")["Min Delay"].sum() / TTC_delay_code["Min Delay"].size()
TTC_delay_code_avg_delaytime.sort_values(ascending=False)


Code
ERHV     289.714286
MUFM     135.333333
MUPR1     76.954545
MRPLB     67.555556
PUTS      66.500000
            ...    
EUCH       3.500000
MRFS       3.000000
TUTD       3.000000
MUNCA      0.500000
MFO        0.000000
Name: Min Delay, Length: 194, dtype: float64

### Number and % for each delay reasons

In [57]:
## The # times that delay happened

TTC_delay_number = TTC_delay.groupby("Code").size().sort_values(ascending=False)
TTC_delay_number.head()

Code
SUDP     3583
MUIR     2097
MUPAA    1981
MUI      1747
EUDO     1343
dtype: int64

In [58]:
## The % times that delay happened

TTC_delay_percet = TTC_delay.groupby("Code")["Day"].count() / TTC_delay.shape[0]*100 
TTC_delay_percet.head()

Code
ERAC    0.043385
ERBO    0.018981
ERCD    0.135579
ERCO    0.048808
ERDB    0.108463
Name: Day, dtype: float64

### The sumed up delay time for each delay reasons

In [29]:
TTC_delay_sumdelaytime = TTC_delay.groupby(["Code"])["Min Delay"].sum().sort_values(ascending=False)
TTC_delay_sumdelaytime.head(10)

Code
SUDP     22818.0
MUI      16147.0
SUUT     15611.0
MUPLB    14303.0
MUPR1    13544.0
MUIR     11900.0
SUO      11476.0
MUPAA     7557.0
EUDO      7035.0
PUOPO     6411.0
Name: Min Delay, dtype: float64

Disorderly Patron; 
Injured or ill Customer (On Train) - Transported; 
Unauthorized at Track Level; 
Fire/Smoke Plan B - Source TTC; 
Priority One - Train in Contact With Person
Injured or ill Customer (On Train) - Medical Aid Refused
Passenger Other
Passenger Assistance Alarm Activated - No Trouble Found
Door Problems - Faulty Equipment
OPTO (COMMS) Train Door Monitoring

### The sumed up numbers for each delay reason

In [31]:
TTC_delay_sumdelaytime = TTC_delay.groupby(["Code"])["Day"].count().sort_values(ascending=False)
TTC_delay_sumdelaytime.head(10)

Code
SUDP     3583
MUIR     2097
MUPAA    1981
MUI      1747
EUDO     1343
PUOPO    1313
SUO      1296
SUUT     1161
MUATC    1041
MUTO     1009
Name: Day, dtype: int64

*Disorderly Patron;
*Injured or ill Customer (On Train) - Medical Aid Refused
*Passenger Assistance Alarm Activated - No Trouble Found
*Injured or ill Customer (On Train) - Transported
*Door Problems - Faulty Equipment
*OPTO (COMMS) Train Door Monitoring
*Passenger Other
*Unauthorized at Track Level
*ATC Project
*Misc. Transportation Other - Employee Non-Chargeable

### The most frequency delay reason for each stations 

In [52]:
## The count here shows number 

TTC_delay_stations_count = TTC_delay.groupby(["Station"]).agg(total_count=('Code', 'count'), max_delay_code=('Code', 'max')).sort_values(by='total_count', ascending=False)

TTC_delay_stations.head(10)

,total_count,max_delay_code
Station,,
FINCH STATION,1615,TUSUP
EGLINTON STATION,1531,TUSUP
KENNEDY BD STATION,1531,TUSUP
KIPLING STATION,1374,TUSUP
BLOOR STATION,985,TUSUP
SHEPPARD WEST STATION,959,TUSUP
MCCOWAN STATION,919,TRTC
WILSON STATION,860,TUSUP
ST GEORGE YUS STATION,836,TUSUP


### Long-term delay and short-term delay 

The short-term delay is determined if the min delay time. 

1). Short term delay if the min gap is within 5 mintes, 

2). Medium term delay if the min delay is 5 < min delay < 10,

3). Long term delay if the min delay is > 10 mins

*TTC might be suggested to take different actions based on different delay type

In [74]:
## Defind the delay type based on the min delay time and add a column for delay type 

def delay_type(gap):
    if gap >= 1 and gap < 5:
        return 'short-term delay'
    elif gap >= 5 and gap <= 10:
        return 'medium-term delay'
    elif gap >= 10:
        return 'long-term delay'
    
TTC_delay['delay_type'] = TTC_delay['Min Delay'].apply(delay_type)

TTC_delay.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_30468\2821185711.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TTC_delay['delay_type'] = TTC_delay['Min Delay'].apply(delay_type)


,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Code description,time_interval,Period,delay_type
Date/Time,,,,,,,,,,,,
2014-01-01 00:21:00,Wednesday,VICTORIA PARK STATION,MUPR1,55.0,60.0,W,BD,5111,Priority One - Train in Contact With Person,00:00 - 01:00,1,long-term delay
2014-01-01 02:06:00,Wednesday,HIGH PARK STATION,SUDP,3.0,7.0,W,BD,5001,Disorderly Patron,02:00 - 03:00,1,short-term delay
2014-01-01 03:10:00,Wednesday,LANSDOWNE STATION,SUDP,3.0,8.0,W,BD,5116,Disorderly Patron,03:00 - 04:00,1,short-term delay
2014-01-01 03:20:00,Wednesday,BLOOR STATION,MUSAN,5.0,10.0,S,YU,5386,Unsanitary Vehicle,03:00 - 04:00,1,medium-term delay
2014-01-01 08:48:00,Wednesday,KIPLING STATION,MUNOA,5.0,NaN,E,BD,0,No Operator Immediately Available - Not E.S.A....,08:00 - 09:00,2,medium-term delay
